In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw02_integration_and_odes_sol.ipynb")

# Homework 2: Integration and Differential Equations
**CHBE 2250 - Modeling and Simulation in Chemical Engineering**

---

**Name:** _[Your Name]_

**Date:** _[Date]_

---

## Instructions

This homework assesses your understanding of numerical integration and solving ordinary differential equations (ODEs), covering material from notebooks 02-05. You are **encouraged to use AI tools** to help you write code, but you must:

1. **Understand all code you submit** - You should be able to explain what each line does
2. **Document your AI usage** - Add a comment indicating which parts were AI-assisted
3. **Complete reflection questions independently** - These require critical thinking and cannot be answered by AI alone

**Estimated time:** 3 hours for proficient students using AI assistance

**Submission:**
- Work through each exercise and run the test cells to check your work
- When complete, save and download your notebook
- Submit the .ipynb file to Gradescope

**Auto-grading:** This notebook uses **otter-grader** for automatic testing. After completing each exercise, you can run the test cell to verify your solution is correct.

**Total Points:** 100 points (4 exercises × 25 points each)

---

## Getting Started

**IMPORTANT:** Run the setup cells below first to install and initialize otter-grader.

In [ ]:
# IMPORTANT: Run this cell first in Google Colab
# This installs required packages

import sys

# Install required packages
!pip install -q numpy scipy matplotlib otter-grader

print("✓ Packages installed successfully!")

In [ ]:
# Initialize otter-grader
import otter
grader = otter.Notebook()

print("✓ Otter-grader is ready!")
print("You can check your work anytime by running: grader.check('exercise_name')")

In [ ]:
# Import libraries needed for all exercises
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad, trapz, cumtrapz, solve_ivp

print("✓ Libraries imported successfully!")

---

## Exercise 1: Heat Exchanger Energy Balance (25 points)

### Background

In a counter-current heat exchanger, hot process fluid transfers heat to cold cooling water. The heat transfer rate varies along the length of the exchanger due to changing temperature differences.

The differential heat transfer is given by:
$$dQ = U \cdot A'(x) \cdot \Delta T(x) \cdot dx$$

where:
- $U$ = overall heat transfer coefficient = 250 W/(m²·K)
- $A'(x)$ = heat transfer area per unit length = $\pi D$ where $D = 0.05$ m (pipe diameter)
- $\Delta T(x)$ = temperature difference between hot and cold fluids at position $x$
- $x$ = position along the exchanger (m)

The total heat transferred is:
$$Q_{total} = \int_0^L U \cdot \pi D \cdot \Delta T(x) \, dx$$

### Problem Setup

You have measured the temperature difference $\Delta T$ at various positions along a 10 m heat exchanger:

| Position x (m) | ΔT (K) |
|----------------|--------|
| 0.0 | 45.0 |
| 2.0 | 38.5 |
| 4.0 | 32.8 |
| 6.0 | 27.5 |
| 8.0 | 22.9 |
| 10.0 | 18.5 |

### Part A: Calculate Total Heat Transfer using Trapezoid Method (10 points)

1. Define arrays for position `x` and temperature difference `deltaT`
2. Calculate the integrand (the function being integrated)
3. Use `np.trapz()` to calculate the total heat transfer in Watts
4. Store the result in a variable named `Q_total_trapz`

In [ ]:
# AI assistance: No

# Given parameters
U = 250  # W/(m^2·K)
D = 0.05  # m

# Measured data
x = np.array([0.0, 2.0, 4.0, 6.0, 8.0, 10.0])  # m
deltaT = np.array([45.0, 38.5, 32.8, 27.5, 22.9, 18.5])  # K

# Calculate the integrand
integrand = U * np.pi * D * deltaT

# Calculate total heat transfer using trapezoid method
Q_total_trapz = np.trapz(integrand, x)

print(f"Total heat transfer (trapezoid method): {Q_total_trapz:.2f} W")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex1a")

### Part B: Cumulative Heat Transfer Profile (8 points)

Calculate and plot how the cumulative heat transferred varies with position along the heat exchanger.

1. Use `cumtrapz()` from `scipy.integrate` to calculate cumulative heat transfer
2. Store the result in variable `Q_cumulative`
3. Create a plot with:
   - x-axis: Position (m)
   - y-axis: Cumulative heat transfer (W)
   - Proper labels and title

In [ ]:
# AI assistance: No

# Calculate cumulative heat transfer
from scipy.integrate import cumtrapz

integrand = U * np.pi * D * deltaT
Q_cumulative = cumtrapz(integrand, x, initial=0)

# Plot cumulative heat transfer
plt.figure(figsize=(8, 5))
plt.plot(x, Q_cumulative, 'b-o', linewidth=2, markersize=6)
plt.xlabel('Position along heat exchanger (m)', fontsize=12)
plt.ylabel('Cumulative heat transfer (W)', fontsize=12)
plt.title('Cumulative Heat Transfer Profile', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Heat transferred in first 6 m: {Q_cumulative[3]:.2f} W")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex1b")

### Part C: Critical Reflection (7 points)

**Answer the following in your own words (do not use AI):**

You used the trapezoid method with only 6 data points spanning 10 meters. 

1. How would having more measurement points (say, every 0.5 m instead of every 2 m) affect the accuracy of your heat transfer calculation?
2. In a real heat exchanger design, the temperature profile might be highly non-linear near the inlet or outlet. How would this affect the reliability of your calculation with sparse data points?
3. What practical engineering considerations would determine how many measurement points you actually use in practice?

**Your reflection answer:**

_[Write your answer here - minimum 4-5 sentences]_

---

## Exercise 2: CSTR with First-Order Reaction (25 points)

### Background

A Continuous Stirred Tank Reactor (CSTR) is used for a liquid-phase reaction where species A converts to product B:
$$A \rightarrow B$$

The reaction is first-order with rate:
$$r_A = -k C_A$$

For a well-mixed CSTR at constant volume and temperature, a mole balance on species A gives:
$$\frac{dC_A}{dt} = \frac{q}{V}(C_{A,in} - C_A) - k C_A$$

where:
- $C_A$ = concentration of A in reactor (mol/L)
- $C_{A,in}$ = inlet concentration of A (mol/L)
- $q$ = volumetric flow rate (L/min)
- $V$ = reactor volume (L)
- $k$ = rate constant (1/min)
- $t$ = time (min)

### Problem Setup

A CSTR has the following parameters:
- Volume: $V = 100$ L
- Flow rate: $q = 10$ L/min
- Inlet concentration: $C_{A,in} = 2.0$ mol/L
- Rate constant: $k = 0.15$ 1/min
- Initial concentration: $C_A(0) = 0$ mol/L (reactor initially filled with pure solvent)

### Part A: Solve for Concentration vs Time (12 points)

1. Define a function `dCAdt(t, CA)` that returns the derivative $dC_A/dt$
2. Use `solve_ivp()` to solve from t=0 to t=60 min
3. Evaluate the solution at 100 time points using `t_eval`
4. Store the solution object as `sol`
5. Plot concentration vs time with proper labels

In [ ]:
# AI assistance: No

# Parameters
V = 100  # L
q = 10  # L/min
CA_in = 2.0  # mol/L
k = 0.15  # 1/min

# Define ODE function
def dCAdt(t, CA):
    """Derivative of concentration in CSTR"""
    CA_val = CA[0]  # Extract value from array
    dCA = (q/V) * (CA_in - CA_val) - k * CA_val
    return [dCA]

# Solve ODE
CA0 = [0.0]  # initial condition
tspan = (0, 60)
t_eval = np.linspace(0, 60, 100)

sol = solve_ivp(dCAdt, tspan, CA0, t_eval=t_eval, method='RK45')

# Plot results
plt.figure(figsize=(8, 5))
plt.plot(sol.t, sol.y[0], 'b-', linewidth=2)
plt.xlabel('Time (min)', fontsize=12)
plt.ylabel('Concentration of A (mol/L)', fontsize=12)
plt.title('CSTR Concentration Dynamics', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Store final concentration for testing
CA_final = sol.y[0, -1]
print(f"Final concentration at t=60 min: {CA_final:.4f} mol/L")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex2a")

### Part B: Find Time to Reach 95% of Steady State (6 points)

The steady-state concentration can be calculated analytically from:
$$C_{A,ss} = \frac{C_{A,in}}{1 + \frac{kV}{q}}$$

1. Calculate $C_{A,ss}$ and store in variable `CA_ss`
2. Calculate 95% of steady state: `CA_target = 0.95 * CA_ss`
3. Use an event function with `solve_ivp()` to find when $C_A$ reaches `CA_target`
4. Store the time in variable `t_95`

In [ ]:
# AI assistance: No

# Calculate steady-state concentration
CA_ss = CA_in / (1 + k*V/q)
CA_target = 0.95 * CA_ss

print(f"Steady-state concentration: {CA_ss:.4f} mol/L")
print(f"Target concentration (95% of SS): {CA_target:.4f} mol/L")

# Define event function
def reach_target(t, CA):
    """Event when CA reaches target value"""
    return CA[0] - CA_target

reach_target.terminal = True  # Stop integration when event occurs
reach_target.direction = 1  # Only detect when crossing from below

# Solve with event
sol_event = solve_ivp(dCAdt, tspan, CA0, events=reach_target, dense_output=True)

# Extract time when target was reached
t_95 = sol_event.t_events[0][0]

print(f"Time to reach 95% of steady state: {t_95:.2f} min")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex2b")

### Part C: Critical Reflection (7 points)

**Answer the following in your own words (do not use AI):**

1. The time to reach 95% of steady state is an important design parameter. How would this time change if you doubled the flow rate $q$ while keeping all other parameters constant? Would it increase or decrease?
2. From a process operation standpoint, what are the trade-offs between operating at high flow rates vs. low flow rates in terms of:
   - Time to reach steady state
   - Conversion of reactant A
   - Reactor size requirements
3. Would you rather have a fast or slow approach to steady state when starting up a chemical plant? Why?

**Your reflection answer:**

_[Write your answer here - minimum 5-6 sentences]_

---

## Exercise 3: Two-Tank Bioreactor System (25 points)

### Background

You are designing a two-stage bioreactor system for producing a pharmaceutical product. The system consists of:
- **Tank 1**: Growth phase - cells multiply but produce little product
- **Tank 2**: Production phase - cells produce the desired protein

Each tank has a biomass concentration $X$ (g/L) and product concentration $P$ (g/L).

### Mass Balance Equations

**Tank 1 (Volume = 500 L):**
$$\frac{dX_1}{dt} = \mu_1 X_1 + \frac{F_{in}}{V_1}(X_{in} - X_1) - \frac{F_{12}}{V_1}X_1$$
$$\frac{dP_1}{dt} = q_1 X_1 + \frac{F_{in}}{V_1}(P_{in} - P_1) - \frac{F_{12}}{V_1}P_1$$

**Tank 2 (Volume = 800 L):**
$$\frac{dX_2}{dt} = \mu_2 X_2 + \frac{F_{12}}{V_2}(X_1 - X_2) - \frac{F_{out}}{V_2}X_2$$
$$\frac{dP_2}{dt} = q_2 X_2 + \frac{F_{12}}{V_2}(P_1 - P_2) - \frac{F_{out}}{V_2}P_2$$

where:
- $\mu_1, \mu_2$ = specific growth rates (1/hr)
- $q_1, q_2$ = specific production rates (g product / g biomass / hr)
- $F_{in}$ = feed flow rate to Tank 1 (L/hr)
- $F_{12}$ = flow rate from Tank 1 to Tank 2 (L/hr)
- $F_{out}$ = outlet flow rate from Tank 2 (L/hr)

### Problem Setup

**Parameters:**
- Tank 1: $V_1 = 500$ L, $\mu_1 = 0.3$ 1/hr, $q_1 = 0.05$ g/(g·hr)
- Tank 2: $V_2 = 800$ L, $\mu_2 = 0.1$ 1/hr, $q_2 = 0.25$ g/(g·hr)
- Flow rates: $F_{in} = 50$ L/hr, $F_{12} = 50$ L/hr, $F_{out} = 50$ L/hr
- Feed: $X_{in} = 1.0$ g/L, $P_{in} = 0$ g/L
- Initial conditions: All concentrations start at 0.1 g/L

### Part A: Solve System of ODEs (15 points)

1. Define a function `dYdt(t, Y)` where `Y = [X1, P1, X2, P2]`
2. The function should return `[dX1dt, dP1dt, dX2dt, dP2dt]`
3. Solve from t=0 to t=50 hours with 200 evaluation points
4. Store solution as `sol_bioreactor`
5. Create two subplots:
   - Top: Biomass concentrations X1 and X2 vs time
   - Bottom: Product concentrations P1 and P2 vs time

In [ ]:
# AI assistance: No

# Parameters
V1 = 500  # L
V2 = 800  # L
mu1 = 0.3  # 1/hr
mu2 = 0.1  # 1/hr
q1 = 0.05  # g/(g*hr)
q2 = 0.25  # g/(g*hr)
F_in = 50  # L/hr
F_12 = 50  # L/hr
F_out = 50  # L/hr
X_in = 1.0  # g/L
P_in = 0.0  # g/L

# Define system of ODEs
def dYdt(t, Y):
    """System of ODEs for two-tank bioreactor"""
    X1, P1, X2, P2 = Y

    # Tank 1 derivatives
    dX1dt = mu1*X1 + (F_in/V1)*(X_in - X1) - (F_12/V1)*X1
    dP1dt = q1*X1 + (F_in/V1)*(P_in - P1) - (F_12/V1)*P1

    # Tank 2 derivatives
    dX2dt = mu2*X2 + (F_12/V2)*(X1 - X2) - (F_out/V2)*X2
    dP2dt = q2*X2 + (F_12/V2)*(P1 - P2) - (F_out/V2)*P2

    return [dX1dt, dP1dt, dX2dt, dP2dt]

# Initial conditions: [X1, P1, X2, P2]
Y0 = [0.1, 0.1, 0.1, 0.1]
tspan = (0, 50)
t_eval = np.linspace(0, 50, 200)

# Solve system
sol_bioreactor = solve_ivp(dYdt, tspan, Y0, t_eval=t_eval, method='RK45')

# Extract solutions for easier plotting
t = sol_bioreactor.t
X1 = sol_bioreactor.y[0]
P1 = sol_bioreactor.y[1]
X2 = sol_bioreactor.y[2]
P2 = sol_bioreactor.y[3]

# Create plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot biomass concentrations
ax1.plot(t, X1, 'b-', linewidth=2, label='Tank 1 (X₁)')
ax1.plot(t, X2, 'r-', linewidth=2, label='Tank 2 (X₂)')
ax1.set_xlabel('Time (hr)', fontsize=11)
ax1.set_ylabel('Biomass Concentration (g/L)', fontsize=11)
ax1.set_title('Biomass Dynamics in Two-Tank System', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot product concentrations
ax2.plot(t, P1, 'b-', linewidth=2, label='Tank 1 (P₁)')
ax2.plot(t, P2, 'r-', linewidth=2, label='Tank 2 (P₂)')
ax2.set_xlabel('Time (hr)', fontsize=11)
ax2.set_ylabel('Product Concentration (g/L)', fontsize=11)
ax2.set_title('Product Dynamics in Two-Tank System', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Store final values for testing
X1_final = X1[-1]
P2_final = P2[-1]
print(f"Final biomass in Tank 1: {X1_final:.4f} g/L")
print(f"Final product in Tank 2: {P2_final:.4f} g/L")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex3a")

### Part B: Product Yield Analysis (3 points)

Calculate the overall product yield of the system:

$$\text{Yield} = \frac{\text{Product output rate}}{\text{Biomass input rate}} = \frac{F_{out} \cdot P_2(t=50)}{F_{in} \cdot X_{in}}$$

Store the result in variable `product_yield` (dimensionless ratio)

In [ ]:
# AI assistance: No

# Calculate product yield
product_yield = (F_out * P2_final) / (F_in * X_in)

print(f"Overall product yield: {product_yield:.4f} g product / g biomass input")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex3b")

### Part C: Critical Reflection (7 points)

**Answer the following in your own words (do not use AI):**

1. Looking at your plots, explain why Tank 1 has higher biomass concentration but Tank 2 has higher product concentration. What does this tell you about the design strategy of separating growth and production phases?

2. The system has equal flow rates ($F_{in} = F_{12} = F_{out} = 50$ L/hr), meaning volumes are constant. If you wanted to increase the final product concentration $P_2$, what parameter changes would you consider? Think about residence times and specific production rates.

3. In real bioreactors, maintaining sterile conditions is critical and expensive. What are the trade-offs of using a two-tank system versus a single larger tank for this application?

**Your reflection answer:**

_[Write your answer here - minimum 6-7 sentences]_

---

## Exercise 4: Damped Oscillations in Level Control (25 points)

### Background

A liquid storage tank has an automatic level control system. When the liquid level deviates from the setpoint, a controller adjusts the inlet valve. However, the controller can cause oscillations if not properly tuned.

The dynamics of the tank level deviation from setpoint can be modeled as a damped harmonic oscillator:

$$\frac{d^2h}{dt^2} + 2\zeta\omega_n\frac{dh}{dt} + \omega_n^2 h = 0$$

where:
- $h(t)$ = deviation from setpoint level (cm)
- $\zeta$ = damping ratio (dimensionless)
- $\omega_n$ = natural frequency (rad/min)

**Damping cases:**
- $\zeta < 1$: Underdamped (oscillatory)
- $\zeta = 1$: Critically damped (fastest return without overshoot)
- $\zeta > 1$: Overdamped (slow return, no overshoot)

### Problem Setup

A disturbance causes the level to be initially 20 cm above setpoint with zero initial velocity:
- Initial conditions: $h(0) = 20$ cm, $\frac{dh}{dt}(0) = 0$ cm/min
- Natural frequency: $\omega_n = 0.5$ rad/min

### Part A: Compare Different Damping Ratios (12 points)

1. Convert the 2nd-order ODE to a system of first-order ODEs using $v = dh/dt$
2. Define function `dXdt(t, X, zeta)` where `X = [h, v]`
3. Solve for three cases: $\zeta = 0.2, 1.0, 2.0$ from t=0 to t=30 min
4. Create a plot showing all three responses on the same graph
5. Store the solution for $\zeta = 0.2$ as `sol_underdamped`

In [ ]:
# AI assistance: No

# Parameters
omega_n = 0.5  # rad/min
h0 = 20  # cm
v0 = 0  # cm/min

# Define ODE system
def dXdt(t, X, zeta):
    """Second-order damped oscillator as system of first-order ODEs"""
    h, v = X
    dhdt = v
    dvdt = -2*zeta*omega_n*v - omega_n**2*h
    return [dhdt, dvdt]

# Initial conditions
X0 = [h0, v0]
tspan = (0, 30)
t_eval = np.linspace(0, 30, 300)

# Solve for three damping ratios
zeta_values = [0.2, 1.0, 2.0]
labels = ['Underdamped (ζ=0.2)', 'Critically damped (ζ=1.0)', 'Overdamped (ζ=2.0)']
colors = ['blue', 'green', 'red']
linestyles = ['-', '--', '-.']

plt.figure(figsize=(10, 6))

for i, zeta in enumerate(zeta_values):
    # Solve ODE for this zeta value using lambda function
    sol = solve_ivp(lambda t, X: dXdt(t, X, zeta), tspan, X0, t_eval=t_eval, method='RK45')

    if zeta == 0.2:
        sol_underdamped = sol  # Save for testing

    # Plot h vs t
    plt.plot(sol.t, sol.y[0], color=colors[i], linestyle=linestyles[i],
             linewidth=2, label=labels[i])

# Add labels, legend, grid
plt.xlabel('Time (min)', fontsize=12)
plt.ylabel('Level Deviation from Setpoint (cm)', fontsize=12)
plt.title('Tank Level Control Response for Different Damping Ratios', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle=':', linewidth=1, alpha=0.5)
plt.tight_layout()
plt.show()

# For testing: get final value for underdamped case
h_final_underdamped = sol_underdamped.y[0, -1]
print(f"Final deviation (underdamped): {h_final_underdamped:.4f} cm")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex4a")

### Part B: Find Oscillation Period (6 points)

For the underdamped case ($\zeta = 0.2$), find the period of oscillation by detecting successive maxima.

1. Define an event function that detects when $\frac{dh}{dt} = 0$ and h is at a maximum
2. Use `solve_ivp()` with the event function to find all maxima
3. Calculate the average period between maxima
4. Store the average period in variable `oscillation_period` (in minutes)

**Theory:** For underdamped oscillations, the period is:
$$T = \frac{2\pi}{\omega_d} = \frac{2\pi}{\omega_n\sqrt{1-\zeta^2}}$$

In [ ]:
# AI assistance: No

# Define event function to detect maxima
def find_maximum(t, X, zeta):
    """Event when velocity crosses zero (dh/dt = 0)"""
    h, v = X
    return v

find_maximum.direction = -1  # Detect when v goes from positive to negative (maximum)

# Solve with event detection
zeta = 0.2
sol_events = solve_ivp(lambda t, X: dXdt(t, X, zeta), tspan, X0,
                       events=lambda t, X: find_maximum(t, X, zeta),
                       dense_output=True, max_step=0.1)

# Extract times of maxima
t_maxima = sol_events.t_events[0]
print(f"Times of maxima: {t_maxima}")

# Calculate periods between consecutive maxima
periods = np.diff(t_maxima)
oscillation_period = np.mean(periods)

print(f"\nAverage oscillation period: {oscillation_period:.4f} min")

# Compare with theoretical value
theoretical_period = 2 * np.pi / (omega_n * np.sqrt(1 - zeta**2))
print(f"Theoretical period: {theoretical_period:.4f} min")
print(f"Percent error: {abs(oscillation_period - theoretical_period)/theoretical_period * 100:.2f}%")

In [ ]:
# TEST CELL - DO NOT DELETE
grader.check("ex4b")

### Part C: Critical Reflection (7 points)

**Answer the following in your own words (do not use AI):**

1. Based on your plots, which damping ratio ($\zeta = 0.2, 1.0,$ or $2.0$) would you choose for a tank level controller in:
   - A tank feeding a critical reactor where level must stabilize quickly?
   - A storage tank where oscillations could cause pump damage?
   Justify your choices.

2. The critically damped case ($\zeta = 1$) is often called "optimal" damping. Why might this be preferred in process control applications, even though it takes longer to reach setpoint than some underdamped cases?

3. In real systems, what physical factors might cause the damping ratio to change over time? How would this affect your control strategy?

**Your reflection answer:**

_[Write your answer here - minimum 6-7 sentences]_

---

## AI Usage Documentation

**Required:** Provide a brief summary (4-6 sentences) of how you used AI tools in completing this homework. Be specific about which exercises AI helped with and how you verified the code worked correctly.

_[Your AI usage summary here]_

---

## Grading Rubric

| Exercise | Code Functionality | Reflection Quality | Total |
|----------|-------------------|-------------------|-------|
| 1 - Heat Exchanger | 18 pts | 7 pts | 25 pts |
| 2 - CSTR | 18 pts | 7 pts | 25 pts |
| 3 - Bioreactor | 18 pts | 7 pts | 25 pts |
| 4 - Level Control | 18 pts | 7 pts | 25 pts |
| **Total** | **72 pts** | **28 pts** | **100 pts** |

**Code is evaluated on:**
- Correctness and functionality
- Proper use of numerical methods
- Clear documentation and comments
- Appropriate visualization

**Reflections are evaluated on:**
- Demonstration of conceptual understanding
- Critical thinking beyond the calculations
- Connection to real engineering applications
- Thoughtful analysis (not AI-generated responses)

---

## Submission

1. Make sure all test cells pass
2. Save your notebook: File > Save
3. Download: File > Download > Download .ipynb
4. Upload the .ipynb file to Gradescope

In [ ]:
# Run this cell to check all tests before submission
import datetime

print("Running all checks...")
print("=" * 50)

checks = ['ex1a', 'ex1b', 'ex2a', 'ex2b', 'ex3a', 'ex3b', 'ex4a', 'ex4b']
results = {}

for check in checks:
    try:
        result = grader.check(check)
        results[check] = "✓ Passed"
        print(f"{check}: ✓ Passed")
    except Exception as e:
        results[check] = f"✗ Failed: {str(e)[:50]}"
        print(f"{check}: ✗ Failed - {str(e)[:50]}")

print("=" * 50)
print(f"\nSummary: {sum(1 for r in results.values() if '✓' in r)}/{len(checks)} checks passed")
print(f"\nTimestamp: {datetime.datetime.now()}")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)